## Chat with Tableau

Tableau's [VizQL Data Service](https://help.tableau.com/current/api/vizql-data-service/en-us/index.html) (aka VDS) provides developers with programmatic access to their Tableau Published Data Sources, allowing them to extend their business semantics for any custom workload or application, including AI Agents. The simple_datasource_qa tool adds VDS to the Langchain framework. This notebook shows you how you can use it to build agents that answer analytical questions grounded on your enterprise semantic models. 

Follow the [tableau-langchain](https://github.com/Tab-SE/tableau_langchain) project for more tools coming soon!


## Requirements
1. python version 3.12.2 or higher
2. A Tableau Cloud or Server environment with at least 1 published data source

Get started by installing and/or importing the required packages

In [ ]:
%pip install langchain-openai

In [ ]:
%pip install langgraph

In [ ]:
%pip install langchain-tableau --upgrade

Note you may need to restart your kernal to use updated packages

## Overview
The initialize_simple_datasource_qa initializes the Langgraph tool called [simple_datasource_qa](https://github.com/Tab-SE/tableau_langchain/blob/3ff9047414479cd55d797c18a78f834d57860761/pip_package/langchain_tableau/tools/simple_datasource_qa.py#L101), which can be used for analytical questions and answers on a Tableau Data Source.

This initializer function:
1. Authenticates to Tableau using Tableau's connected-app framework for JWT-based authentication. All the required variables must defined at runtime or as environment variables.
2. Asynchronously queries for the field metadata of the target datasource specified in the datasource_luid variable
3. Grounds on the metadata of the target datasource to transform natural language questions into the json-formatted query payload required to make VDS /query-datasource requests 
4. Executes a POST request to VDS
5. Formats and returns the results in a structured response

In [ ]:
#langchain and langgraph package imports
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.prebuilt import create_react_agent

#langchain_tableau imports
#from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa
from experimental.tools.simple_datasource_qa import initialize_simple_datasource_qa

## Authentication Variables
You can declare your environment variables explicitly, as shown in several cases in this cookbook. However, ff these parameters are not provided, the simple_datasource_qa tool will attempt to automatically read them from environment variables.

For the Data Source that you choose, make sure you've updated the VizqlDataApiAccess permission in Tableau to allow the VDS API to access that Data Source via REST. More info [here](https://help.tableau.com/current/server/en-us/permissions_capabilities.htm#data-sources
). 

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

tableau_server = os.getenv('TABLEAU_DOMAIN') #replace with your Tableau server name
tableau_site = os.getenv('TABLEAU_SITE') #replace with your Tableau site
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID') #a JWT client ID (obtained through Tableau's admin UI)
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID') #a JWT secret ID (obtained through Tableau's admin UI)
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET') #a JWT secret ID (obtained through Tableau's admin UI)
tableau_api_version = '3.21' #the current Tableau REST API Version
tableau_user = os.getenv('TABLEAU_USER') #replace with the username querying the target Tableau Data Source

# For this cookbook we are connecting to the Superstore dataset that comes by default with every Tableau server
datasource_luid = os.getenv('DATASOURCE_LUID') #the target data source for this Tool

# Add variables to control LLM models for the Agent and Tools
os.getenv("OPENAI_API_KEY") #set an your model API key as an environment variable
tooling_llm_model = os.getenv("TOOLING_MODEL") #set the LLM model for the Agent
print(f"Tooling LLM Model: {tooling_llm_model}")

## Langchain Example
First, we'll initlialize the LLM of our choice. Next, we initialize our tool for chatting with tableau data sources and store it in a variable called analyze_datasource. Finally, we define an agent using Langchain legacy initialize_agent constructor and invoke it with a query related to the target data source. 

In [ ]:
from IPython.display import display, Markdown

# Initialize an LLM 
llm = ChatOpenAI(model='o4-mini', temperature=0)

# Initalize simple_datasource_qa for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model)

# load the List of Tools to be used by the Agent. In this case we will just load our data source Q&A tool.
tools = [ analyze_datasource ]

tableauHeadlessAgent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS, # Use OpenAI's function calling
    verbose = False)

# Run the agent
response = tableauHeadlessAgent.invoke("which school are available in the dataset?")
response
#display(Markdown(response['output'])) #display a nicely formatted answer for successful generations

In [ ]:
def env_vars_simple_datasource_qa(
    domain=None,
    site=None,
    jwt_client_id=None,
    jwt_secret_id=None,
    jwt_secret=None,
    tableau_api_version=None,
    tableau_user=None,
    datasource_luid=None,
    model_provider=None,
    tooling_llm_model=None
):
    """
    Retrieves Tableau configuration from environment variables if not provided as arguments.

    Args:
        domain (str, optional): Tableau domain
        site (str, optional): Tableau site
        jwt_client_id (str, optional): JWT client ID
        jwt_secret_id (str, optional): JWT secret ID
        jwt_secret (str, optional): JWT secret
        tableau_api_version (str, optional): Tableau API version
        tableau_user (str, optional): Tableau user
        datasource_luid (str, optional): Datasource LUID
        tooling_llm_model (str, optional): Tooling LLM model

    Returns:
        dict: A dictionary containing all the configuration values
    """
    # Load environment variables before accessing them
    load_dotenv()

    config = {
        'domain': domain if isinstance(domain, str) and domain else os.environ['TABLEAU_DOMAIN'],
        'site': site or os.environ['TABLEAU_SITE'],
        'jwt_client_id': jwt_client_id or os.environ['TABLEAU_JWT_CLIENT_ID'],
        'jwt_secret_id': jwt_secret_id or os.environ['TABLEAU_JWT_SECRET_ID'],
        'jwt_secret': jwt_secret or os.environ['TABLEAU_JWT_SECRET'],
        'tableau_api_version': tableau_api_version or os.environ['TABLEAU_API_VERSION'],
        'tableau_user': tableau_user or os.environ['TABLEAU_USER'],
        'datasource_luid': datasource_luid or os.environ['DATASOURCE_LUID'],
        'model_provider': model_provider or os.environ['MODEL_PROVIDER'],
        'tooling_llm_model': tooling_llm_model or os.environ['TOOLING_MODEL']
    }

    return config

## Langgraph Example
This example uses the updated langgraph agent constructor class to achieve the same outcome. 

In [ ]:
from IPython.display import display, Markdown

# Initalize simple_datasource_qa for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model)

tools = [analyze_datasource]

model = ChatOpenAI(model='gpt-4o', temperature=0)

tableauAgent = create_react_agent(model, tools)

# Run the agent
messages = tableauAgent.invoke({"messages": [("human",'Rank schools by their average score in Writing where the score is greater than 499, showing their charter numbers.')]})
messages
#display(Markdown(messages['messages'][4].content)) #display a nicely formatted answer for successful generations

In [ ]:
display(Markdown(messages['messages'][2].content)) #display a nicely formatted answer for successful generations